In [4]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
import pandas as pd
from datetime import date, timedelta
from matplotlib import pyplot as plt
import matplotlib as mpl

In [5]:
df = spark.read.csv('bank-full.csv',sep=",",header = True,inferSchema = True)

NameError: name 'spark' is not defined

In [3]:
df = spark.read.csv('bank-full.csv',sep=",",header = True,inferSchema = True)

NameError: name 'spark' is not defined

In [1]:
df.show(1,vertical = True)

NameError: name 'df' is not defined

## __Parent Class__

In [149]:
from abc import ABC, abstractmethod
class QualityRule(ABC):
    def __init__(self, df, field:str, dtype_expected:str, operator:str, threshold, values:list, min_value, max_value,
                key, df2 , field2, key2):
        super().__init__()
        self.data = df
        self.field = field
        self.dtype_expected = dtype_expected
        self.operator = operator
        self.threshold = threshold
        self.values = values
        self.min_value = min_value
        self.max_value = max_value
        self.key = key
        self.df2 = df2
        self.field2 = field2
        self.key2 = key2
        self.population = self._count_population()
        self.errors_df = self._errors_df()
        self.errors = self._count_errors()
        self.compliance = round(((self.population - self.errors)/self.population) * 100,2)
        
    @abstractmethod
    def _count_population():
        pass
    
    @abstractmethod
    def _errors_df():
        pass
    
    @abstractmethod
    def _count_errors():
        pass

## __Child Class__

### V_3_1

In [56]:
class V_3_1(QualityRule):

    def __init__(self, df, field):
         super().__init__(df, field, dtype_expected = '', operator = '', threshold = '', values = '',min_value = '', max_value = '',
                         key = '',df2 = '', field2 = '', key2 = '')
        
    def _count_population(self):
        return self.data.count()
    
    def _errors_df(self):
        return self.data.filter(F.col(self.field).isNull())
    
    def _count_errors(self):
        return self.errors_df.count()

In [57]:
a = V_3_1(gregorian, "day_of_week_id")

In [58]:
a.population

73414

### V_3_2

In [59]:
class V_3_2(QualityRule):    
    
    def __init__(self, df, field, dtype_expected):
        super().__init__(df, field, dtype_expected, operator = '', threshold = '', values = '', min_value = '', max_value = '',
                        key = '',df2 = '', field2 = '', key2 = '')
        
    def _count_population(self):
        return len(self.data.select(self.field).dtypes[0][1].split())
    
    def _errors_df(self):
        return '-'
        
    def _count_errors(self):
        if self.data.select(self.field).dtypes[0][1] == self.dtype_expected:
            return 0
        else:
            return 1

In [76]:
b = V_3_2(gregorian, "day_of_week_id", "int")

In [77]:
b.errors

0

### V_3_3

#### V_3_3_C

In [62]:
class V_3_3_C(QualityRule):    
    
    def __init__(self, df, field, operator, threshold):
        super().__init__(df = df, field = field, dtype_expected = '', operator = operator, threshold = threshold,
                         values = '', min_value = '', max_value = '', key = '', df2 = '', field2 = '', key2 = '')
        
    def _count_population(self):
        return self.data.count()
    
    def _errors_df(self):
        
        if self.operator == '<=':
            return self.data.filter(F.col(self.field) <= self.threshold)
        elif self.operator == '<':
            return self.data.filter(F.col(self.field) < self.threshold)
        elif self.operator == '>':
            return self.data.filter(F.col(self.field) > self.threshold)
        elif self.operator == '>=':
            return self.data.filter(F.col(self.field) >= self.threshold)
        elif self.operator == '==':
            return self.data.filter(F.col(self.field) == self.threshold)
        elif self.operator == '!=':
            return self.data.filter(F.col(self.field) != self.threshold)
        
    def _count_errors(self):
        return self.errors_df.count()    

In [63]:
d = V_3_3_C(trans, "debit_card_trans_event_amount", "<", 0)

In [64]:
d.compliance

100.0

#### V_3_3_D

In [68]:
class V_3_3_D(QualityRule):    
    
    def __init__(self, df, field:str, values:list):
        super().__init__(df = df, field = field, dtype_expected = '', operator = '', threshold = '', values = values,
                         min_value = '', max_value = '', key = '', df2 = '', field2 = '', key2 = '')
        
    def _count_population(self):
        return self.data.count()
    
    def _errors_df(self):
        return self.data.filter(F.col(self.field).isin(self.values))
        
    def _count_errors(self):
        return self.errors_df.count()    

In [69]:
e = V_3_3_D(trans, "debit_card_type", [2])

In [74]:
e.errors

1559641

### V_3_4

In [93]:
class V_3_4(QualityRule):    
    
    def __init__(self, df, field:str, min_value, max_value):
        super().__init__(df = df, field = field, dtype_expected = '', operator = '', threshold = '',
                         values = '', min_value = min_value, max_value = max_value, key = '', df2 = '', field2 = '', key2 = '')
        
    def _count_population(self):
        return self.data.count()
    
    def _errors_df(self):
        return self.data.filter((F.col(self.field) < self.min_value) | (F.col(self.field) > self.max_value))
        
    def _count_errors(self):
        return self.errors_df.count()    

In [94]:
f = V_3_4(trans, "debit_card_type", 1,5)

In [96]:
f.errors

119210

### V_3_5

#### V_3_5_E

In [105]:
class V_3_5_E(QualityRule):    
    
    def __init__(self, df, field:str, values):
        super().__init__(df = df, field = field, dtype_expected = '', operator = '', threshold = '',
                         values = values, min_value = '', max_value = '', key = '', df2 = '', field2 = '', key2 = '')
        
    def _count_population(self):
        return self.data.count()
    
    def _errors_df(self):
        return self.data.filter(~F.col(self.field).isin(self.values))
        
    def _count_errors(self):
        return self.errors_df.count()    

In [106]:
V35E = V_3_5_E(trans, "currency_id", ["ARS","USD"])

In [108]:
V35E.compliance

89.12

### V_3_6

In [136]:
class V_3_6(QualityRule):    
    
    def __init__(self, df, field:str):
        super().__init__(df = df, field = field, dtype_expected = '', operator = '', threshold = '',
                         values = '', min_value = '', max_value = '', key = '', df2 = '', field2 = '', key2 = '')
        
    def _count_population(self):
        return self.data.count()
    
    def _errors_df(self):
        return self.data.withColumn('blanks', F.length(F.regexp_replace(self.field, '[^ ]+', ''))).filter(F.col('blanks')!= 0)
        
    def _count_errors(self):
        return self.errors_df.count()    

In [137]:
V36 = V_3_6(trans,'currency_id')

In [141]:
V36.errors

0

### C_4_2

In [142]:
class C_4_2(QualityRule):
    
    def __init__(self, df, field):
        super().__init__(df, field, dtype_expected='', operator = '', threshold = '', values = '', min_value = '', max_value = '',
                        key = '',df2 = '', field2 = '', key2 = '')
        
    def _count_population(self):
        return self.data.count()
    
    def _errors_df(self):
        return self.data.select(self.field).groupBy(self.field).count().withColumn('errores',F.col('count')-1)
    
    def _count_errors(self):
        return self.errors_df.select(F.sum('errores')).collect()[0][0]

In [143]:
C42 = C_4_2(gregorian, "day_of_week_id")

In [144]:
C42.errors

73407

In [152]:
class C_4_3(QualityRule):
    
    def __init__(self, df, field, key, df2, field2, key2):
        super().__init__(df, field, dtype_expected='', operator = '', threshold = '', values = '', min_value = '', max_value = '',
                        key = key, df2 = df2, field2 = field2, key2 = key2)
        
    def _count_population(self):
        return self.data.count()
    
    def _errors_df(self):
        return self.data.filter( F.col(self.field1+'_1') != F.col(self.field1+'_2')).count()
    
    def _count_errors(self):
        return self.errors_df.count()

In [ ]:
C43 = C_4_3()